In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import coint
from colorama import Fore, Style
import warnings
warnings.filterwarnings('ignore')

## 1. Data Loading

Load full cubic spline datasets for all metals.

In [2]:
# Define metals
metal_names = ['cobalt', 'copper', 'lithium', 'nickel']

# Load full datasets
print("="*60)
print("LOADING DATASETS")
print("="*60 + "\n")

metal_data = {}
for metal in metal_names:
    df = pd.read_csv(f'data/ALL_{metal}_prices_cubic_spline.csv', 
                     parse_dates=['Date'], index_col='Date')
    
    # Remove LISAME series if present
    if 'LISAME' in df.columns:
        df = df.drop(columns=['LISAME'])
        print(f"{metal.capitalize()}: LISAME excluded")
    
    metal_data[metal.capitalize()] = df
    print(f"{metal.capitalize()}: {df.shape[0]} rows, {df.shape[1]} series")
    print(f"  Columns: {df.columns.tolist()}")
    print()

LOADING DATASETS

Cobalt: 3876 rows, 6 series
  Columns: ['CODALY', 'COLMEX', 'COLMEA', 'COWUXI', 'COCOMX', 'COSMMS']

Copper: 2342 rows, 6 series
  Columns: ['CUDALY', 'CUCOMX', 'CULMEX', 'CUSMMG', 'CUSHFE', 'CUETFC']

Lithium: LISAME excluded
Lithium: 2202 rows, 6 series
  Columns: ['LIDALY', 'LICOMX', 'LILAMC', 'LIEALC', 'LIEABG', 'LILMEX']

Nickel: 2857 rows, 6 series
  Columns: ['NIDALY', 'NILMEX', 'NIETFN', 'NISHFE', 'NIWUXI', 'NIINDA']



## 2. Compute Log-Returns

Convert prices to log-returns for cointegration testing.

In [3]:
print("="*60)
print("COMPUTING LOG-RETURNS")
print("="*60 + "\n")

metal_returns = {}
for metal in metal_names:
    df = metal_data[metal.capitalize()].copy()
    # Compute log-returns
    df_returns = np.log(df / df.shift(1)).dropna(how='all')
    metal_returns[metal.capitalize()] = df_returns
    print(f"{metal.capitalize()} log-returns: {df_returns.shape[0]} rows, {df_returns.shape[1]} series")

print("\nLog-returns will be used for cointegration analysis.")

COMPUTING LOG-RETURNS

Cobalt log-returns: 3875 rows, 6 series
Copper log-returns: 2341 rows, 6 series
Lithium log-returns: 2201 rows, 6 series
Nickel log-returns: 2856 rows, 6 series

Log-returns will be used for cointegration analysis.


## 2b. Compute Regular Returns (for comparison)

Calculate regular percentage returns as well.

In [4]:
print("="*60)
print("COMPUTING REGULAR RETURNS")
print("="*60 + "\n")

metal_returns_regular = {}
for metal in metal_names:
    df = metal_data[metal.capitalize()].copy()
    # Compute regular returns (percentage change)
    df_returns = df.pct_change().dropna(how='all')
    metal_returns_regular[metal.capitalize()] = df_returns
    print(f"{metal.capitalize()} regular returns: {df_returns.shape[0]} rows, {df_returns.shape[1]} series")

print("\nRegular returns (pct_change) available for alternative analysis.")

COMPUTING REGULAR RETURNS

Cobalt regular returns: 3875 rows, 6 series
Copper regular returns: 2341 rows, 6 series
Lithium regular returns: 2201 rows, 6 series
Nickel regular returns: 2856 rows, 6 series

Regular returns (pct_change) available for alternative analysis.


## 3. Configuration

Define series pairs and ticker mappings for each metal.

In [5]:
# Ticker mappings for LaTeX output (maps column names to display names)
ticker_mappings = {
    'Cobalt': {
        'CODALY': 'CODALY',
        'COLMEX': 'COLMEX',
        'COLME3': 'COLME3',
        'COLMEA': 'COLMEA',
        'COWUXI': 'COWUXI',
        'COCOMX': 'COCOMX',
        'COSMMS': 'COSMMS'
    },
    'Copper': {
        'CUDALY': 'CUDALY',
        'CULMEX': 'CULMEX',
        'CULME3': 'CULME3',
        'CULMEA': 'CULMEA',
        'CUWUXI': 'CUWUXI',
        'CUCOMX': 'CUCOMX',
        'CUSMMS': 'CUSMMS'
    },
    'Lithium': {
        'LIDALY': 'LIDALY',
        'LILMEX': 'LILMEX',
        'LILME3': 'LILME3',
        'LILMEA': 'LILMEA',
        'LIWUXI': 'LIWUXI',
        'LICOMX': 'LICOMX',
        'LISMMS': 'LISMMS'
    },
    'Nickel': {
        'NIDALY': 'NIDALY',
        'NILMEX': 'NILMEX',
        'NILME3': 'NILME3',
        'NILMEA': 'NILMEA',
        'NIWUXI': 'NIWUXI',
        'NICOMX': 'NICOMX',
        'NISMMS': 'NISMMS'
    }
}

# Define series pairs for testing (adjust based on actual column names)
# This is a template - will be constructed dynamically for each metal
def get_series_pairs(columns):
    """Generate all unique pairwise combinations of series."""
    pairs = []
    cols = [c for c in columns if c != 'LISAME']  # Exclude LISAME
    for i, col1 in enumerate(cols):
        for col2 in cols[i+1:]:
            pairs.append((col1, col2))
    return pairs

## 4. Helper Functions

In [6]:
def run_cointegration_tests(df, pairs, metal_name):
    """
    Run Engle-Granger cointegration tests for specified pairs.
    
    Parameters:
    - df: DataFrame with series as columns
    - pairs: List of (series1, series2) tuples
    - metal_name: Name of the metal for reporting
    
    Returns:
    - List of (series1, series2, score, p_value) tuples
    """
    results = []
    
    # Get list of all series involved
    all_series = set()
    for s1, s2 in pairs:
        all_series.add(s1)
        all_series.add(s2)
    
    # Drop rows with NaN in any of the required series
    df_clean = df[list(all_series)].dropna()
    
    print(f"\n{Fore.YELLOW}{metal_name} - Testing {len(pairs)} pairs{Style.RESET_ALL}")
    print(f"Data range: {df_clean.index.min()} to {df_clean.index.max()}")
    print(f"Observations: {len(df_clean)}\n")
    
    for series1, series2 in pairs:
        try:
            score, p_value, _ = coint(df_clean[series1], df_clean[series2])
            results.append((series1, series2, score, p_value))
        except Exception as e:
            print(f"Error testing {series1} vs {series2}: {e}")
            results.append((series1, series2, np.nan, np.nan))
    
    return results


def print_results_table(results, metal_name):
    """Print cointegration results as a formatted console table."""
    print(f"\n{Fore.YELLOW}{metal_name} Cointegration Test Results:{Style.RESET_ALL}")
    print(f"{Fore.CYAN}{'Series 1':<15}{'Series 2':<15}{'Score':<15}{'P-Value':<15}{Style.RESET_ALL}")
    
    for series1, series2, score, p_value in results:
        if pd.notna(p_value):
            p_value_color = Fore.GREEN if p_value < 0.05 else Style.RESET_ALL
            print(f"{series1:<15}{series2:<15}{Fore.GREEN}{score:<15.4f}{p_value_color}{p_value:<15.4f}{Style.RESET_ALL}")
        else:
            print(f"{series1:<15}{series2:<15}{'N/A':<15}{'N/A':<15}")


def print_latex_table(results, metal_name, ticker_map):
    """Generate LaTeX table for cointegration results."""
    print(f"\n{Fore.CYAN}LaTeX Table for {metal_name}:{Style.RESET_ALL}\n")
    
    print("\\begin{table}[ht]")
    print("\\centering")
    print("{\\small")
    print("\\begin{tabular}{llrr}")
    print("\\toprule")
    print("Series 1 & Series 2 & Score & P-Value \\\\")
    print("\\midrule")
    
    for series1, series2, score, p_value in results:
        if pd.notna(p_value):
            # Get ticker symbols from mapping
            ticker1 = ticker_map.get(series1, series1.replace("_", "\\_"))
            ticker2 = ticker_map.get(series2, series2.replace("_", "\\_"))
            
            # Format p-value with green color if significant
            p_value_str = f"\\textcolor{{green}}{{{p_value:.4f}}}" if p_value < 0.05 else f"{p_value:.4f}"
            
            print(f"{ticker1} & {ticker2} & {score:.4f} & {p_value_str} \\\\")
    
    print("\\bottomrule")
    print("\\end{tabular}")
    print("}")
    print(f"\\caption{{Cointegration Test Results - {metal_name}}}")
    print(f"\\label{{tab:cointegration_{metal_name.lower()}}}")
    print("\\end{table}")
    print()

## 5. Cobalt Cointegration Analysis

In [7]:
metal = 'Cobalt'
df = metal_returns[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, metal)

# Print results
print_results_table(results, metal)

# Print LaTeX table
print_latex_table(results, metal, ticker_mappings[metal])

COBALT - COINTEGRATION ANALYSIS

Cobalt - Testing 15 pairs
Data range: 2023-05-05 00:00:00 to 2025-07-21 00:00:00
Observations: 661


Cobalt Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
CODALY         COLMEX         -7.1299        0.0000         
CODALY         COLMEA         -7.5760        0.0000         
CODALY         COWUXI         -7.0237        0.0000         
CODALY         COCOMX         -7.7798        0.0000         
CODALY         COSMMS         -9.1209        0.0000         
COLMEX         COLMEA         -6.6686        0.0000         
COLMEX         COWUXI         -6.5433        0.0000         
COLMEX         COCOMX         -6.8198        0.0000         
COLMEX         COSMMS         -7.6917        0.0000         
COLMEA         COWUXI         -14.8070       0.0000         
COLMEA         COCOMX         -14.8173       0.0000         
COLMEA         COSMMS         -14.8048       0.0000         
COWUXI         COCOMX         -5.4870 

## 6. Copper Cointegration Analysis

In [8]:
metal = 'Copper'
df = metal_returns[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, metal)

# Print results
print_results_table(results, metal)

# Print LaTeX table
print_latex_table(results, metal, ticker_mappings[metal])

COPPER - COINTEGRATION ANALYSIS

Copper - Testing 15 pairs
Data range: 2024-03-07 00:00:00 to 2025-10-31 00:00:00
Observations: 431


Copper Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
CUDALY         CUCOMX         -11.1320       0.0000         
CUDALY         CULMEX         -11.1803       0.0000         
CUDALY         CUSMMG         -23.0396       0.0000         
CUDALY         CUSHFE         -23.3436       0.0000         
CUDALY         CUETFC         -22.8000       0.0000         
CUCOMX         CULMEX         -6.9915        0.0000         
CUCOMX         CUSMMG         -23.1646       0.0000         
CUCOMX         CUSHFE         -23.4873       0.0000         
CUCOMX         CUETFC         -23.3622       0.0000         
CULMEX         CUSMMG         -24.2880       0.0000         
CULMEX         CUSHFE         -24.8332       0.0000         
CULMEX         CUETFC         -25.4034       0.0000         
CUSMMG         CUSHFE         -10.4654

## 7. Lithium Cointegration Analysis

In [9]:
metal = 'Lithium'
df = metal_returns[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, metal)

# Print results
print_results_table(results, metal)

# Print LaTeX table
print_latex_table(results, metal, ticker_mappings[metal])

LITHIUM - COINTEGRATION ANALYSIS

Lithium - Testing 15 pairs
Data range: 2023-10-03 00:00:00 to 2025-10-31 00:00:00
Observations: 555


Lithium Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
LIDALY         LICOMX         -10.5847       0.0000         
LIDALY         LILAMC         -10.5775       0.0000         
LIDALY         LIEALC         -10.8170       0.0000         
LIDALY         LIEABG         -9.2211        0.0000         
LIDALY         LILMEX         -10.5821       0.0000         
LICOMX         LILAMC         -20.2697       0.0000         
LICOMX         LIEALC         -20.3362       0.0000         
LICOMX         LIEABG         -20.3610       0.0000         
LICOMX         LILMEX         -20.2954       0.0000         
LILAMC         LIEALC         -6.0600        0.0000         
LILAMC         LIEABG         -6.0836        0.0000         
LILAMC         LILMEX         -5.5716        0.0000         
LIEALC         LIEABG         -13.8

## 8. Nickel Cointegration Analysis

In [10]:
metal = 'Nickel'
df = metal_returns[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, metal)

# Print results
print_results_table(results, metal)

# Print LaTeX table
print_latex_table(results, metal, ticker_mappings[metal])

NICKEL - COINTEGRATION ANALYSIS

Nickel - Testing 15 pairs
Data range: 2023-03-23 00:00:00 to 2025-07-21 00:00:00
Observations: 697


Nickel Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
NIDALY         NILMEX         -10.5333       0.0000         
NIDALY         NIETFN         -8.9208        0.0000         
NIDALY         NISHFE         -8.1306        0.0000         
NIDALY         NIWUXI         -9.1672        0.0000         
NIDALY         NIINDA         -7.7471        0.0000         
NILMEX         NIETFN         -21.8489       0.0000         
NILMEX         NISHFE         -22.8307       0.0000         
NILMEX         NIWUXI         -11.5555       0.0000         
NILMEX         NIINDA         -15.3209       0.0000         
NIETFN         NISHFE         -19.5486       0.0000         
NIETFN         NIWUXI         -7.7610        0.0000         
NIETFN         NIINDA         -7.7887        0.0000         
NISHFE         NIWUXI         -15.3189

## 9. Cointegration Analysis on Regular Returns

Repeat cointegration tests using regular percentage returns instead of log-returns.

### 9.1 Cobalt (Regular Returns)

In [11]:
metal = 'Cobalt'
df = metal_returns_regular[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS (REGULAR RETURNS)")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, f"{metal} (Regular Returns)")

# Print results
print_results_table(results, f"{metal} (Regular Returns)")

# Print LaTeX table
print_latex_table(results, f"{metal} (Regular Returns)", ticker_mappings[metal])

COBALT - COINTEGRATION ANALYSIS (REGULAR RETURNS)

Cobalt (Regular Returns) - Testing 15 pairs
Data range: 2023-05-05 00:00:00 to 2025-11-18 00:00:00
Observations: 747


Cobalt (Regular Returns) Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
CODALY         COLMEX         -7.0958        0.0000         
CODALY         COLMEA         -7.3397        0.0000         
CODALY         COWUXI         -7.6436        0.0000         
CODALY         COCOMX         -7.6403        0.0000         
CODALY         COSMMS         -9.2083        0.0000         
COLMEX         COLMEA         -6.4908        0.0000         
COLMEX         COWUXI         -6.4269        0.0000         
COLMEX         COCOMX         -6.6409        0.0000         
COLMEX         COSMMS         -7.4617        0.0000         
COLMEA         COWUXI         -15.4578       0.0000         
COLMEA         COCOMX         -15.4999       0.0000         
COLMEA         COSMMS         -15.4795       

### 9.2 Copper (Regular Returns)

In [12]:
metal = 'Copper'
df = metal_returns_regular[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS (REGULAR RETURNS)")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, f"{metal} (Regular Returns)")

# Print results
print_results_table(results, f"{metal} (Regular Returns)")

# Print LaTeX table
print_latex_table(results, f"{metal} (Regular Returns)", ticker_mappings[metal])

COPPER - COINTEGRATION ANALYSIS (REGULAR RETURNS)

Copper (Regular Returns) - Testing 15 pairs
Data range: 2024-03-07 00:00:00 to 2025-12-31 00:00:00
Observations: 474


Copper (Regular Returns) Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
CUDALY         CUCOMX         -11.6825       0.0000         
CUDALY         CULMEX         -7.1045        0.0000         
CUDALY         CUSMMG         -24.3204       0.0000         
CUDALY         CUSHFE         -24.3562       0.0000         
CUDALY         CUETFC         -23.9726       0.0000         
CUCOMX         CULMEX         -7.4761        0.0000         
CUCOMX         CUSMMG         -24.4041       0.0000         
CUCOMX         CUSHFE         -24.3950       0.0000         
CUCOMX         CUETFC         -24.5527       0.0000         
CULMEX         CUSMMG         -25.5102       0.0000         
CULMEX         CUSHFE         -25.4731       0.0000         
CULMEX         CUETFC         -26.6452       

### 9.3 Lithium (Regular Returns)

In [13]:
metal = 'Lithium'
df = metal_returns_regular[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS (REGULAR RETURNS)")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, f"{metal} (Regular Returns)")

# Print results
print_results_table(results, f"{metal} (Regular Returns)")

# Print LaTeX table
print_latex_table(results, f"{metal} (Regular Returns)", ticker_mappings[metal])

LITHIUM - COINTEGRATION ANALYSIS (REGULAR RETURNS)

Lithium (Regular Returns) - Testing 15 pairs
Data range: 2023-10-03 00:00:00 to 2025-12-09 00:00:00
Observations: 573


Lithium (Regular Returns) Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
LIDALY         LICOMX         -10.7965       0.0000         
LIDALY         LILAMC         -10.7895       0.0000         
LIDALY         LIEALC         -11.0235       0.0000         
LIDALY         LIEABG         -9.3915        0.0000         
LIDALY         LILMEX         -10.7956       0.0000         
LICOMX         LILAMC         -20.5423       0.0000         
LICOMX         LIEALC         -20.6223       0.0000         
LICOMX         LIEABG         -20.6289       0.0000         
LICOMX         LILMEX         -20.5375       0.0000         
LILAMC         LIEALC         -6.0256        0.0000         
LILAMC         LIEABG         -6.0554        0.0000         
LILAMC         LILMEX         -6.0331     

### 9.4 Nickel (Regular Returns)

In [14]:
metal = 'Nickel'
df = metal_returns_regular[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS (REGULAR RETURNS)")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, f"{metal} (Regular Returns)")

# Print results
print_results_table(results, f"{metal} (Regular Returns)")

# Print LaTeX table
print_latex_table(results, f"{metal} (Regular Returns)", ticker_mappings[metal])

NICKEL - COINTEGRATION ANALYSIS (REGULAR RETURNS)

Nickel (Regular Returns) - Testing 15 pairs
Data range: 2023-03-23 00:00:00 to 2025-11-18 00:00:00
Observations: 783


Nickel (Regular Returns) Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
NIDALY         NILMEX         -10.9405       0.0000         
NIDALY         NIETFN         -9.4798        0.0000         
NIDALY         NISHFE         -8.6722        0.0000         
NIDALY         NIWUXI         -9.7730        0.0000         
NIDALY         NIINDA         -8.3164        0.0000         
NILMEX         NIETFN         -10.9813       0.0000         
NILMEX         NISHFE         -24.1645       0.0000         
NILMEX         NIWUXI         -7.6761        0.0000         
NILMEX         NIINDA         -8.0790        0.0000         
NIETFN         NISHFE         -21.0787       0.0000         
NIETFN         NIWUXI         -20.9720       0.0000         
NIETFN         NIINDA         -21.0650       

## 10. Cointegration Analysis on Raw Prices

Perform cointegration tests directly on price levels (not returns).

### 10.1 Cobalt (Raw Prices)

In [15]:
metal = 'Cobalt'
df = metal_data[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS (RAW PRICES)")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, f"{metal} (Raw Prices)")

# Print results
print_results_table(results, f"{metal} (Raw Prices)")

# Print LaTeX table
print_latex_table(results, f"{metal} (Raw Prices)", ticker_mappings[metal])

COBALT - COINTEGRATION ANALYSIS (RAW PRICES)

Cobalt (Raw Prices) - Testing 15 pairs
Data range: 2023-05-04 00:00:00 to 2025-07-21 00:00:00
Observations: 662


Cobalt (Raw Prices) Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
CODALY         COLMEX         -5.0591        0.0001         
CODALY         COLMEA         -1.9685        0.5449         
CODALY         COWUXI         -4.4133        0.0017         
CODALY         COCOMX         -5.3865        0.0000         
CODALY         COSMMS         -2.8035        0.1646         
COLMEX         COLMEA         -2.0305        0.5127         
COLMEX         COWUXI         -4.4505        0.0015         
COLMEX         COCOMX         -5.3927        0.0000         
COLMEX         COSMMS         -2.8588        0.1477         
COLMEA         COWUXI         0.0480         0.9870         
COLMEA         COCOMX         0.0168         0.9863         
COLMEA         COSMMS         -0.6662        0.9502         

### 10.2 Copper (Raw Prices)

In [16]:
metal = 'Copper'
df = metal_data[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS (RAW PRICES)")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, f"{metal} (Raw Prices)")

# Print results
print_results_table(results, f"{metal} (Raw Prices)")

# Print LaTeX table
print_latex_table(results, f"{metal} (Raw Prices)", ticker_mappings[metal])

COPPER - COINTEGRATION ANALYSIS (RAW PRICES)

Copper (Raw Prices) - Testing 15 pairs
Data range: 2024-03-06 00:00:00 to 2025-10-31 00:00:00
Observations: 432


Copper (Raw Prices) Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
CUDALY         CUCOMX         -8.7784        0.0000         
CUDALY         CULMEX         -2.9905        0.1125         
CUDALY         CUSMMG         -3.2927        0.0557         
CUDALY         CUSHFE         -3.2277        0.0654         
CUDALY         CUETFC         -2.6202        0.2296         
CUCOMX         CULMEX         -1.9284        0.5655         
CUCOMX         CUSMMG         -3.4358        0.0385         
CUCOMX         CUSHFE         -3.3699        0.0458         
CUCOMX         CUETFC         -2.7437        0.1843         
CULMEX         CUSMMG         -3.2049        0.0691         
CULMEX         CUSHFE         -3.1947        0.0708         
CULMEX         CUETFC         -2.3824        0.3332         

### 10.3 Lithium (Raw Prices)

In [17]:
metal = 'Lithium'
df = metal_data[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS (RAW PRICES)")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, f"{metal} (Raw Prices)")

# Print results
print_results_table(results, f"{metal} (Raw Prices)")

# Print LaTeX table
print_latex_table(results, f"{metal} (Raw Prices)", ticker_mappings[metal])

LITHIUM - COINTEGRATION ANALYSIS (RAW PRICES)

Lithium (Raw Prices) - Testing 15 pairs
Data range: 2023-10-02 00:00:00 to 2025-10-31 00:00:00
Observations: 556


Lithium (Raw Prices) Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
LIDALY         LICOMX         -3.4181        0.0404         
LIDALY         LILAMC         -3.6633        0.0204         
LIDALY         LIEALC         -3.1993        0.0700         
LIDALY         LIEABG         -2.9570        0.1208         
LIDALY         LILMEX         -2.6390        0.2223         
LICOMX         LILAMC         -3.2272        0.0655         
LICOMX         LIEALC         -1.5782        0.7300         
LICOMX         LIEABG         -2.3522        0.3478         
LICOMX         LILMEX         -6.6503        0.0000         
LILAMC         LIEALC         -1.3416        0.8168         
LILAMC         LIEABG         -2.3098        0.3687         
LILAMC         LILMEX         -2.1977        0.4256      

### 10.4 Nickel (Raw Prices)

In [18]:
metal = 'Nickel'
df = metal_data[metal]

# Generate all pairwise combinations
pairs = get_series_pairs(df.columns)

print(f"{'='*60}")
print(f"{metal.upper()} - COINTEGRATION ANALYSIS (RAW PRICES)")
print(f"{'='*60}")

# Run tests
results = run_cointegration_tests(df, pairs, f"{metal} (Raw Prices)")

# Print results
print_results_table(results, f"{metal} (Raw Prices)")

# Print LaTeX table
print_latex_table(results, f"{metal} (Raw Prices)", ticker_mappings[metal])

NICKEL - COINTEGRATION ANALYSIS (RAW PRICES)

Nickel (Raw Prices) - Testing 15 pairs
Data range: 2023-03-22 00:00:00 to 2025-07-21 00:00:00
Observations: 698


Nickel (Raw Prices) Cointegration Test Results:
Series 1       Series 2       Score          P-Value        
NIDALY         NILMEX         -7.9996        0.0000         
NIDALY         NIETFN         -2.5946        0.2390         
NIDALY         NISHFE         -2.7415        0.1850         
NIDALY         NIWUXI         -2.5266        0.2674         
NIDALY         NIINDA         -4.2790        0.0028         
NILMEX         NIETFN         -3.4666        0.0355         
NILMEX         NISHFE         -3.3429        0.0491         
NILMEX         NIWUXI         -2.2202        0.4140         
NILMEX         NIINDA         -5.2228        0.0001         
NIETFN         NISHFE         -2.5000        0.2791         
NIETFN         NIWUXI         -2.7457        0.1836         
NIETFN         NIINDA         -2.4106        0.3198         

## 11. Summary

**Methodology:**
- Engle-Granger cointegration test performed on log-returns
- Tests all unique pairwise combinations within each metal
- LISAME series excluded from analysis

**Interpretation:**
- **P-value < 0.05** (highlighted in green): Statistically significant cointegration at 5% level
- Lower p-values indicate stronger evidence of cointegration
- Cointegrated series tend to move together in the long run despite short-term deviations

**Note:** Cointegration is tested within each metal only, not across different metals.